# 파이프라인 엔드투엔드 테스트 노트북

이 노트북은 새로 리팩토링된 파이프라인(LlamaParse + OpenAI Embeddings + Hybrid Search)이 여러 PDF 문서를 엔드투엔드(End-to-End)로 처리하고, 그 결과가 데이터베이스에 잘 저장되며 검색 가능한지 확인합니다.

**시작하기 전:**
1.  `.env` 파일에 `LLAMA_CLOUD_API_KEY`와 `OPENAI_API_KEY`가 올바르게 설정되어 있는지 확인하세요.
2.  PostgreSQL 데이터베이스 Docker 컨테이너가 실행 중인지 확인하세요. (프로젝트 루트에서 `docker-compose up -d`)

In [8]:
# 1. 필요한 모듈 임포트
import os
from pathlib import Path
from dotenv import load_dotenv
import json
import numpy as np
from tqdm import tqdm # Jupyter Notebook용 tqdm

from pipeline import test_search
from db_handler import drop_tables, init_tables, DBHandler
from pdf_filter import get_all_announcement_pdfs
from document_parser import parse_pdf, ParsedDocument
from table_processor import process_tables_in_document
from chunker import create_chunks_from_elements
from embedder import embed_chunks
from db_loader import load_announcement_to_db, load_chunks_to_db

# 환경 변수 로드 (노트북 환경에서 명시적으로 로드)
load_dotenv(dotenv_path='./.env', override=True)

print("모든 모듈 임포트 및 환경 변수 로드 완료")

모든 모듈 임포트 및 환경 변수 로드 완료


In [9]:
# 2. 데이터베이스 초기화 및 테이블 생성
print("데이터베이스 테이블 삭제...")
drop_tables()
print("데이터베이스 테이블 생성...")
init_tables()
print("DB 초기화 및 테이블 생성 완료")

데이터베이스 테이블 삭제...
테이블 삭제 완료
데이터베이스 테이블 생성...
테이블 생성 및 FTS 설정 완료
DB 초기화 및 테이블 생성 완료


In [10]:
# 3. 테스트에 사용할 PDF 파일 선택 및 파이프라인 실행

# --- 설정 시작 --- #
num_pdfs_to_process = 5  # 테스트할 PDF 파일의 개수 (임대 공고에서 선택)
# --- 설정 끝 --- #

all_pdfs = get_all_announcement_pdfs()
selected_pdfs = []

# 임대 공고 PDF를 우선적으로 선택
if all_pdfs['lease']:
    selected_pdfs.extend(all_pdfs['lease'][:num_pdfs_to_process])

if not selected_pdfs:
    raise Exception(f"테스트할 임대 공고 PDF가 {num_pdfs_to_process}개 충분하지 않습니다. 'data/LH_lease_서울.경기' 폴더를 확인하세요.")

print(f"선택된 테스트 PDF: {len(selected_pdfs)}개")
print("--- 파이프라인 처리 시작 ---")

processed_count = 0
total_chunks_processed = 0

for external_id, file_path in tqdm(selected_pdfs, desc="PDF 처리 중"):
    print(f"처리 중: {file_path.name} (ID: {external_id})")
    try:
        # 4. PDF 파싱
        parsed_doc = parse_pdf(file_path, external_id)
        
        # 5. 테이블 자연어 변환 및 청크 생성
        processed_elements = process_tables_in_document(parsed_doc.elements)
        chunks = create_chunks_from_elements(processed_elements, external_id)
        
        # 6. 임베딩 생성
        chunks_with_embeddings = embed_chunks(chunks)
        
        # 7. DB에 적재
        meta_data = {'title': file_path.name, 'category': '임대', 'file_name': file_path.name}
        announcement_id = load_announcement_to_db(external_id, meta_data)
        load_chunks_to_db(chunks_with_embeddings, announcement_id)
        
        processed_count += 1
        total_chunks_processed += len(chunks_with_embeddings)
        print(f"  완료: {file_path.name} ({len(chunks_with_embeddings)}개 청크 저장)")

    except Exception as e:
        print(f"  오류 발생 ({file_path.name}): {e}")

print("--- 파이프라인 처리 완료 ---")
print(f"총 {processed_count}개 파일 처리 성공, 총 {total_chunks_processed}개 청크 저장")

# DB 상태 확인
print("DB에서 공고 및 청크 수 확인...")
with DBHandler() as cursor:
    cursor.execute("SELECT COUNT(*) FROM announcements")
    ann_count = cursor.fetchone()[0]
    cursor.execute("SELECT COUNT(*) FROM document_chunks")
    chunk_count = cursor.fetchone()[0]
print(f"announcements 테이블: {ann_count}개, document_chunks 테이블: {chunk_count}개")

임대 공고 PDF: 341개
분양 공고 PDF: 63개
총 공고 PDF: 404개
선택된 테스트 PDF: 5개
--- 파이프라인 처리 시작 ---


처리 중: (LH_lease_272)_[정정공고]화성시_행복주택_입주자격완화_예비입주자_모집공고문(2024.12.30).pdf (ID: LH_lease_272)
Started parsing the file under job_id 3ccf2f79-ba1c-4cf0-9703-f93d298fff47
OpenAI 임베딩 API 호출 중... (255개 텍스트)
임베딩 모델 초기화: OpenAIEmbeddings (text-embedding-3-small)
모델 초기화 완료
임베딩 생성 완료.


255개 청크 삽입 완료
  완료: (LH_lease_272)_[정정공고]화성시_행복주택_입주자격완화_예비입주자_모집공고문(2024.12.30).pdf (255개 청크 저장)
처리 중: (LH_lease_369)_(정정)파주운정3A24BL영구임대추가모집입주자모집공고문.pdf (ID: LH_lease_369)
Started parsing the file under job_id d28658d1-b55e-4155-bac1-725781afa6eb
OpenAI 임베딩 API 호출 중... (163개 텍스트)
임베딩 생성 완료.


163개 청크 삽입 완료
  완료: (LH_lease_369)_(정정)파주운정3A24BL영구임대추가모집입주자모집공고문.pdf (163개 청크 저장)
처리 중: (LH_lease_1149)_광명시국민임대주택예비입주자모집(2025.07.07공고).pdf (ID: LH_lease_1149)
Started parsing the file under job_id a1768d11-8017-4230-b72e-29154e6c10d6
OpenAI 임베딩 API 호출 중... (209개 텍스트)
임베딩 생성 완료.


209개 청크 삽입 완료
  완료: (LH_lease_1149)_광명시국민임대주택예비입주자모집(2025.07.07공고).pdf (209개 청크 저장)
처리 중: (LH_lease_998)_★25년2차신혼·신생아매입임대Ⅰ_입주자모집공고문(정정).pdf (ID: LH_lease_998)
Started parsing the file under job_id 6af25573-4c95-4d01-8236-3c4158804979
OpenAI 임베딩 API 호출 중... (94개 텍스트)
임베딩 생성 완료.


94개 청크 삽입 완료
  완료: (LH_lease_998)_★25년2차신혼·신생아매입임대Ⅰ_입주자모집공고문(정정).pdf (94개 청크 저장)
처리 중: (LH_lease_699)_20250509의왕포일창업지원예비자공고.pdf (ID: LH_lease_699)
Started parsing the file under job_id 23f05f3c-cd08-4dcd-9f49-abf02fab1ef2
OpenAI 임베딩 API 호출 중... (195개 텍스트)
임베딩 생성 완료.



PDF 처리 중: 100%|██████████| 5/5 [01:13<00:00, 14.74s/it]

195개 청크 삽입 완료
  완료: (LH_lease_699)_20250509의왕포일창업지원예비자공고.pdf (195개 청크 저장)
--- 파이프라인 처리 완료 ---
총 5개 파일 처리 성공, 총 916개 청크 저장
DB에서 공고 및 청크 수 확인...
announcements 테이블: 5개, document_chunks 테이블: 916개


In [13]:
# 4. 최종 하이브리드 검색 테스트
print("--- 최종 하이브리드 검색 테스트 ---")
test_query = "신청자격"

print(f"검색 쿼리: {test_query}")
results = test_search(test_query, top_k=5)

if results:
    print("하이브리드 검색 결과:")
    for i, row in enumerate(results):
        chunk_id, chunk_text, elem_type, table_ctx, title, region, category, similarity = row
        print(f"--- 결과 #{i+1} (유사도: {similarity:.4f}) ---")
        print(f"  타입: {elem_type} / 출처: {title}")
        if table_ctx:
            print(f"  테이블 컨텍스트: {table_ctx}")
        print("[내용]")
        print(chunk_text)
else:
    print("검색 결과가 없습니다.")

--- 최종 하이브리드 검색 테스트 ---
검색 쿼리: 신청자격

검색 쿼리: 신청자격
----------------------------------------
하이브리드 검색 결과:
--- 결과 #1 (유사도: 0.5586) ---
  타입: text / 출처: (LH_lease_1149)_광명시국민임대주택예비입주자모집(2025.07.07공고).pdf
[내용]
신청자격
- 모집공고일 현재 관계법령에서 정한 국민임대주택 입주자격을 갖춘 자가 신청할 수 있습니다. [4. 신청자격 참조]
- 이 주택의 당첨자(예비입주자 포함)및 계약자는 모집 공고일 기준 세대구성원 전원 포함하여 입주자 모집 공고일로부터 임대차계약 종료일까지 무주택이어야 합니다.
- 신청 이후 신청자격에 대하여 부적격 사유가 발견되는 경우 이에 대한 소명의무는 신청자에게 있으니, 소명기간 내에 객관적인 증명서류를 갖추어 이의신청을 하여야 하며, 기한 내에 이의신청을 하지 아니하는 경우에는 부적격 사유에 대하여 이의가 없는 것으로 간주되어 예비입주자 당첨에서 제외합니다.
- 주택소유여부 확인 및 판정은 ’주택공급에 관한 규칙 제52조 및 제53조‘에 따라 처리합니다.
신청방법
- 인터넷신청 또는 현장방문 신청 방법을 반드시 확인한 후 신청하시기 바랍니다. [7. 신청방법 참조]
- 본 모집공고문의 모집대상 주택에 대하여 1세대 1주택 신청을 원칙으로 하며, 중복 신청하는 경우 전부 무효처리 합니다. * 배우자 세대분리는 중복입주로서 본 임대주택 입주가 불가합니다.
--- 결과 #2 (유사도: 0.4615) ---
  타입: text / 출처: (LH_lease_369)_(정정)파주운정3A24BL영구임대추가모집입주자모집공고문.pdf
[내용]
## 3. 신청자격

입주자 모집공고일(2025.02.27.) 현재 파주시에 주민등록이 등재된 성년자인 무주택세대구성원으로서 아래의 소득 및 자산보유 기준, 기타 법에 정한 요건을 충족하고 입주자격제한(불법양도·전대)에 해당하지 않는 사람 (단, 모집공고일로부터 